**Setup and Environment Configuration**

In [1]:
# System & Python packages
# This block installs necessary system-level libraries and Python packages
# required for audio processing and running the Flask server.

# Install system libraries required by madmom and pydub for audio processing.
# libfftw3-dev: Fast Fourier Transform library, crucial for signal processing.
# libsndfile1: Library for reading/writing sound files.
# ffmpeg: Essential for pydub to handle various audio formats (e.g., MP3, M4A).
!sudo apt-get install -y libfftw3-dev libsndfile1 ffmpeg

# Install Python packages using pip.
# madmom: The primary library for music information retrieval (chord detection).
# pydub: For audio manipulation, especially converting various formats to WAV.
# librosa: Another audio analysis library (though madmom is primary here).
# soundfile: Provides audio file I/O capabilities.
# numpy: Fundamental package for numerical computing in Python.
# plotly: (Potentially for visualization, though not directly used in the API logic here).
# Flask: The web framework used to create the API server.
# pyngrok: Allows exposing the local Flask server to the internet via a public URL.
!pip install madmom pydub librosa soundfile numpy plotly Flask pyngrok

# Patch deprecated NumPy aliases
# Recent versions of NumPy have deprecated direct aliases like np.float, np.int, etc.
# This patch ensures compatibility with older libraries (like some versions of madmom)
# that might still use these deprecated aliases, preventing runtime errors.
import numpy as np
if not hasattr(np, 'float'): np.float = float
if not hasattr(np, 'int'): np.int = int
if not hasattr(np, 'complex'): np.complex = complex
if not hasattr(np, 'bool'): np.bool = bool
if not hasattr(np, 'object'): np.object = object

# Patch madmom's outdated import for MutableSequence
# Some versions of madmom might use an outdated import path for MutableSequence
# from the 'collections' module. This sed command updates the import to
# 'collections.abc.MutableSequence', which is the correct path in newer Python versions,
# resolving potential ImportError issues.
!sed -i 's/from collections import MutableSequence/from collections.abc import MutableSequence/' /usr/local/lib/python3.11/dist-packages/madmom/processors.py


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libfftw3-bin libfftw3-double3 libfftw3-long3 libfftw3-quad3 libfftw3-single3
Suggested packages:
  libfftw3-doc
The following NEW packages will be installed:
  libfftw3-bin libfftw3-dev libfftw3-double3 libfftw3-long3 libfftw3-quad3
  libfftw3-single3
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,654 kB of archives.
After this operation, 24.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-double3 amd64 3.3.8-2ubuntu8 [770 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-long3 amd64 3.3.8-2ubuntu8 [335 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-quad3 amd64 3.3.8-2ubuntu8 [614 

/tmp/ipython-input-3380907567.py:31: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'object'): np.object = object


**Flask Backend for Chord Analysis**

In [ ]:
# Flask Backend for Chord Analysis

# This block sets up the Flask web server, defines audio processing functions
# for chord detection, creates the API endpoint, and configures ngrok
# to make the server accessible from the internet.

import os # For file system operations (e.g., creating/deleting temporary files).
import mimetypes # For guessing file types (though less critical after fix).
import tempfile # For creating secure temporary files.
from flask import Flask, request, jsonify # Flask components for web server.
from pyngrok import ngrok # For creating a public tunnel to the Flask app.
from google.colab import userdata # For securely accessing Colab secrets (like ngrok auth token).

# Ensure madmom and other core audio processing libraries are imported.
# A warning is printed if any of these critical imports fail, indicating
# a potential installation issue from the first code block.
try:
    from pydub import AudioSegment # For loading and converting audio files.
    from madmom.features import CNNChordFeatureProcessor, CRFChordRecognitionProcessor # Madmom's chord detection models.
    from madmom.audio.chroma import DeepChromaProcessor # Madmom's chroma feature extraction.
    import numpy as np # For numerical operations, especially with audio data.
except ImportError:
    print("Warning: madmom, pydub, or numpy not fully installed. Check pip output.")

# Initialize the Flask application.
app = Flask(__name__)

# --- Audio Processing Functions ---

def convert_to_wav(input_path):
    """
    Converts an audio file from its original format to WAV format.

    This function uses pydub to load the input audio file (which is expected
    to have its original extension) and exports it to a new temporary WAV file.
    This WAV conversion is often necessary as madmom typically works best with WAV.

    Args:
        input_path (str): The path to the input audio file (e.g., '/tmp/audio.mp3').

    Returns:
        str: The path to the newly created temporary WAV file.

    Raises:
        ValueError: If the input audio format is unsupported or conversion fails.
    """
    print(f"DEBUG(Colab): convert_to_wav called with input_path: {input_path}")

    try:
        # pydub intelligently infers the audio format from the file extension
        # or file header.
        audio = AudioSegment.from_file(input_path)
        print(f"DEBUG(Colab): AudioSegment.from_file successful for {input_path}")
    except Exception as e:
        print(f"DEBUG(Colab): Error loading audio file {input_path} with pydub: {e}")
        raise ValueError(f"Unsupported or invalid audio format (pydub error): {e}")

    # Create a temporary file with a .wav suffix for the output.
    temp_wav_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    wav_path = temp_wav_file.name
    temp_wav_file.close() # Close the file handle immediately so export can write to it.

    try:
        # Export the loaded audio segment to the temporary WAV file.
        audio.export(wav_path, format="wav")
        print(f"DEBUG(Colab): Audio exported to WAV: {wav_path}")
    except Exception as e:
        print(f"DEBUG(Colab): Error exporting audio to WAV: {e}")
        # Ensure cleanup of the partially created WAV file even if export fails.
        if os.path.exists(wav_path):
            os.remove(wav_path)
        raise ValueError(f"Failed to convert to WAV: {e}")

    return wav_path

def detect_chords_and_notes(audio_path):
    """
    Detects chords and their constituent notes in an audio file using madmom.

    This function first converts the audio to WAV, then applies madmom's
    CNN-based chord feature extraction and CRF-based chord recognition.
    It also extracts chroma features to identify individual notes present
    within each detected chord segment.

    Args:
        audio_path (str): The path to the input audio file (can be various formats).

    Returns:
        list: A list of dictionaries, each containing 'start_time', 'end_time',
              'chord' label, and 'detected_notes' for each segment.

    Raises:
        ValueError: If audio conversion fails.
        RuntimeError: If madmom processors are not loaded.
        Exception: For any other errors during madmom processing.
    """
    wav_path = None
    try:
        # Convert the input audio file (which might be MP3, M4A, etc.) to WAV.
        wav_path = convert_to_wav(audio_path)
        print(f"DEBUG(Colab): Starting madmom processing on WAV: {wav_path}")

        # Ensure that madmom components are successfully imported and available.
        if 'CNNChordFeatureProcessor' not in globals() or \
           'CRFChordRecognitionProcessor' not in globals() or \
           'DeepChromaProcessor' not in globals():
            raise RuntimeError("Madmom processors not loaded. Check installation and previous output.")

        # 1. Chord Feature Extraction: Uses a Convolutional Neural Network (CNN)
        #    to extract chord-related features from the audio.
        chord_features = CNNChordFeatureProcessor()(wav_path)
        # 2. Chord Recognition: Applies a Conditional Random Field (CRF) model
        #    to the extracted features to recognize chord labels over time.
        chords = CRFChordRecognitionProcessor()(chord_features)

        # 3. Chroma Feature Extraction: Extracts chroma features (pitch class profiles)
        #    which represent the energy of each of the 12 pitch classes (C, C#, D, etc.).
        chroma = DeepChromaProcessor()(wav_path)
        fps = 10 # Frames per second for chroma features (madmom's default for DeepChroma).

        # Define standard note names for mapping chroma features to notes.
        NOTE_NAMES = ['C', 'C#', 'D', 'D#', 'E', 'F',
                      'F#', 'G', 'G#', 'A', 'A#', 'B']

        output = [] # List to store the final structured chord results.
        # Iterate through each detected chord segment.
        for onset, offset, label in chords:
            start_frame = int(onset * fps)
            end_frame = int(offset * fps)

            # Basic validation for chroma segment boundaries.
            if start_frame >= len(chroma): continue # Skip if start is beyond chroma data.
            if end_frame > len(chroma): end_frame = len(chroma) # Clamp end to chroma data length.
            if start_frame >= end_frame: continue # Skip if segment is invalid.

            # Extract the chroma features for the current chord segment.
            segment_chroma = chroma[start_frame:end_frame]

            # Calculate the average chroma vector for the segment.
            # Handles empty segments gracefully (though should be prevented by checks above).
            if segment_chroma.size == 0:
                avg_chroma = np.zeros(12) # If no data, assume no notes.
            else:
                avg_chroma = np.mean(segment_chroma, axis=0) # Average across frames.

            # Identify individual notes present in the chord based on a threshold.
            # If a pitch class's average energy is above this threshold, it's considered present.
            threshold = 0.2
            notes = [NOTE_NAMES[i] for i, val in enumerate(avg_chroma) if val > threshold]

            # Append the structured result for the current chord segment.
            output.append({
                'start_time': round(float(onset), 2), # Start time rounded to 2 decimal places.
                'end_time': round(float(offset), 2),   # End time rounded to 2 decimal places.
                'chord': label,                        # The recognized chord label (e.g., 'C major').
                'detected_notes': notes                # List of individual notes detected within the chord.
            })
        print("DEBUG(Colab): madmom processing complete.")
        return output
    except Exception as e:
        print(f"DEBUG(Colab): Error during chord detection: {e}")
        raise # Re-raise the exception to be caught by the Flask route for proper error response.
    finally:
        # Ensure the temporary WAV file is cleaned up after processing, regardless of success or failure.
        if wav_path and os.path.exists(wav_path):
            os.remove(wav_path)
            print(f"DEBUG(Colab): Cleaned up temporary WAV file: {wav_path}")

# --- Flask API Endpoint ---

@app.route('/analyze_audio', methods=['POST'])
def analyze_audio():
    """
    Flask API endpoint for analyzing uploaded audio files.

    Expects a POST request with a 'audio_file' multipart form data field.
    It saves the uploaded file temporarily, processes it for chord detection,
    and returns the results as JSON. Handles various error conditions.
    """
    print("DEBUG(Colab): /analyze_audio endpoint hit.")
    # Check if 'audio_file' is present in the request.
    if 'audio_file' not in request.files:
        print("DEBUG(Colab): 'audio_file' not found in request.files. Returning 400.")
        return jsonify({'error': 'No audio_file part in the request'}), 400

    audio_file = request.files['audio_file']
    # Check if a file was actually selected (filename not empty).
    if audio_file.filename == '':
        print("DEBUG(Colab): audio_file.filename is empty. Returning 400.")
        return jsonify({'error': 'No selected file'}), 400

    if audio_file:
        # Get the original file extension from the uploaded filename.
        original_extension = os.path.splitext(audio_file.filename)[1]
        # Create a temporary file with the original extension. This is crucial
        # for pydub to correctly identify the input format.
        temp_input_file = tempfile.NamedTemporaryFile(suffix=original_extension, delete=False)

        # Read the uploaded audio data and write it to the temporary file.
        audio_data = audio_file.read()
        temp_input_file.write(audio_data)
        temp_input_file.close() # Close the file handle to ensure data is flushed to disk.

        print(f"DEBUG(Colab): Received file: '{audio_file.filename}', "
              f"MIME type from request: '{audio_file.mimetype}', "
              f"Uploaded size: {len(audio_data)} bytes")
        print(f"DEBUG(Colab): Saved to temporary path: {temp_input_file.name}")

        try:
            # Verify the size of the saved temporary file on disk.
            saved_file_size = os.path.getsize(temp_input_file.name)
            print(f"DEBUG(Colab): Saved temp file size on disk: {saved_file_size} bytes")

            if saved_file_size == 0:
                print("DEBUG(Colab): Warning: Temporary file is empty after saving. Possible upload issue.")
                raise ValueError("Received empty file.")

            # Pass the temporary file path (which now has the correct extension)
            # to the chord detection processing function.
            results = detect_chords_and_notes(temp_input_file.name)
            print("DEBUG(Colab): Audio analysis successful. Returning 200.")
            # Return a JSON response indicating success and the analysis results.
            return jsonify({'status': 'success', 'results': results}), 200
        except ValueError as ve:
            # Handle specific ValueError exceptions (e.g., unsupported audio format).
            print(f"DEBUG(Colab): Specific ValueError during processing: {ve}. Returning 400.")
            return jsonify({'error': str(ve)}), 400
        except Exception as e:
            # Catch any other unexpected exceptions during processing.
            print(f"DEBUG(Colab): General Exception during processing: {e}. Returning 500.")
            import traceback
            traceback.print_exc() # Print full Python stack trace for debugging.
            return jsonify({'error': f'Audio processing failed: {e}'}), 500
        finally:
            # Ensure the temporary input file is cleaned up after processing.
            if os.path.exists(temp_input_file.name):
                os.remove(temp_input_file.name)
                print(f"DEBUG(Colab): Cleaned up temporary input file: {temp_input_file.name}")
    print("DEBUG(Colab): Unexpected end of /analyze_audio route.")
    return jsonify({'error': 'Something went wrong on the server'}), 500

# --- ngrok Setup ---
# This section configures and starts the ngrok tunnel, making the Flask server
# accessible from the public internet.

try:
    # Attempt to retrieve the ngrok authentication token from Google Colab secrets.
    # It's highly recommended to store your ngrok token as a Colab secret named 'NGROK_AUTH_TOKEN'.
    ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token) # Set the auth token for persistent tunnels.
        print("ngrok auth token loaded from Colab secrets.")
    else:
        print("NGROK_AUTH_TOKEN not found in Colab secrets. Tunnel might be temporary/unreliable.")
except Exception as e:
    print(f"Error loading ngrok auth token from Colab secrets: {e}. Tunnel might be temporary/unreliable.")

# Start the ngrok tunnel on port 5000 (the default Flask port).
# This returns a public URL that the Flutter app can use to connect.
public_url = ngrok.connect(5000)
print(f"Flask App running on: {public_url}")

# Run the Flask application.
# host='0.0.0.0' makes the server accessible from outside localhost (necessary for ngrok).
# port=5000 is the port ngrok is tunneling to.
app.run(host='0.0.0.0', port=5000)


ngrok auth token loaded from Colab secrets.
Flask App running on: NgrokTunnel: "https://be1bed5ad73f.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


DEBUG(Colab): /analyze_audio endpoint hit.
DEBUG(Colab): Received file: 'C:\Users\Eberechukwu Ajayi\Downloads\121Selah_Home.mp3', MIME type from request: 'application/octet-stream', Uploaded size: 9709850 bytes
DEBUG(Colab): Saved to temporary path: /tmp/tmpiomb44f7.mp3
DEBUG(Colab): Saved temp file size on disk: 9709850 bytes
DEBUG(Colab): convert_to_wav called with input_path: /tmp/tmpiomb44f7.mp3
DEBUG(Colab): AudioSegment.from_file successful for /tmp/tmpiomb44f7.mp3
DEBUG(Colab): Audio exported to WAV: /tmp/tmpdqgct10a.wav
DEBUG(Colab): Starting madmom processing on WAV: /tmp/tmpdqgct10a.wav


INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 17:46:53] "POST /analyze_audio HTTP/1.1" 200 -


DEBUG(Colab): madmom processing complete.
DEBUG(Colab): Cleaned up temporary WAV file: /tmp/tmpdqgct10a.wav
DEBUG(Colab): Audio analysis successful. Returning 200.
DEBUG(Colab): Cleaned up temporary input file: /tmp/tmpiomb44f7.mp3
DEBUG(Colab): /analyze_audio endpoint hit.
DEBUG(Colab): Received file: 'chord_testing_file.wav', MIME type from request: 'application/octet-stream', Uploaded size: 5816364 bytes
DEBUG(Colab): Saved to temporary path: /tmp/tmp2p7f047x.wav
DEBUG(Colab): Saved temp file size on disk: 5816364 bytes
DEBUG(Colab): convert_to_wav called with input_path: /tmp/tmp2p7f047x.wav
DEBUG(Colab): AudioSegment.from_file successful for /tmp/tmp2p7f047x.wav
DEBUG(Colab): Audio exported to WAV: /tmp/tmpr0pxsem5.wav
DEBUG(Colab): Starting madmom processing on WAV: /tmp/tmpr0pxsem5.wav


INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 18:01:52] "POST /analyze_audio HTTP/1.1" 200 -


DEBUG(Colab): madmom processing complete.
DEBUG(Colab): Cleaned up temporary WAV file: /tmp/tmpr0pxsem5.wav
DEBUG(Colab): Audio analysis successful. Returning 200.
DEBUG(Colab): Cleaned up temporary input file: /tmp/tmp2p7f047x.wav
DEBUG(Colab): /analyze_audio endpoint hit.
DEBUG(Colab): Received file: 'C:\Users\Eberechukwu Ajayi\Downloads\chord_testing_file.wav', MIME type from request: 'application/octet-stream', Uploaded size: 5816364 bytes
DEBUG(Colab): Saved to temporary path: /tmp/tmphekk8pyt.wav
DEBUG(Colab): Saved temp file size on disk: 5816364 bytes
DEBUG(Colab): convert_to_wav called with input_path: /tmp/tmphekk8pyt.wav
DEBUG(Colab): AudioSegment.from_file successful for /tmp/tmphekk8pyt.wav
DEBUG(Colab): Audio exported to WAV: /tmp/tmpcvfwb689.wav
DEBUG(Colab): Starting madmom processing on WAV: /tmp/tmpcvfwb689.wav


INFO:werkzeug:127.0.0.1 - - [29/Jul/2025 18:25:04] "POST /analyze_audio HTTP/1.1" 200 -


DEBUG(Colab): madmom processing complete.
DEBUG(Colab): Cleaned up temporary WAV file: /tmp/tmpcvfwb689.wav
DEBUG(Colab): Audio analysis successful. Returning 200.
DEBUG(Colab): Cleaned up temporary input file: /tmp/tmphekk8pyt.wav
